In [54]:
import pandas as pd
import re
import numpy as np
from datetime import datetime
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [55]:
train = pd.read_csv("/content/drive/MyDrive/train.csv")
test = pd.read_csv("/content/drive/MyDrive/test_x.csv")

<ipython-input-55-b4125895c607>:1: DtypeWarning: Columns (13,15,19,30,32,37,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("/content/drive/MyDrive/train.csv")


In [56]:
def extract_year(date_str):

    match = re.search(r'(\d{4})', str(date_str))
    if match:
        return int(match.group(1))
    return None

def extract_numbers(text):
    if isinstance(text, str):
        numbers = re.findall(r'\d+', text)
        return int(max(numbers, key=int)) if numbers else 0
    return 0

def clean_text(text):
    if pd.isna(text):
        return None
    return str(text).lower().strip()

In [57]:
train['Dogum Yili'] = train['Dogum Tarihi'].apply(extract_year)
test['Dogum Yili'] = test['Dogum Tarihi'].apply(extract_year)
train['Basvuru Yasi'] = train['Basvuru Yili'] - train['Dogum Yili']
test['Basvuru Yasi'] = test['Basvuru Yili'] - test['Dogum Yili']


In [58]:
sehirler = [
    "Adana", "Adıyaman", "Afyonkarahisar", "Ağrı", "Aksaray", "Amasya", "Ankara", "Antalya",
    "Ardahan", "Artvin", "Aydın", "Balıkesir", "Bartın", "Batman", "Bayburt", "Bilecik", "Bingöl",
    "Bitlis", "Bolu", "Burdur", "Bursa", "Çanakkale", "Çankırı", "Çorum", "Denizli", "Diyarbakır",
    "Düzce", "Edirne", "Elazığ", "Erzincan", "Erzurum", "Eskişehir", "Gaziantep", "Giresun", "Gümüşhane",
    "Hakkari", "Hatay", "Iğdır", "Isparta", "İstanbul", "İzmir", "Kahramanmaraş", "Karabük", "Karaman",
    "Kars", "Kastamonu", "Kayseri", "Kilis", "Kırıkkale", "Kırklareli", "Kırşehir", "Kocaeli", "Konya",
    "Kütahya", "Malatya", "Manisa", "Mardin", "Mersin", "Muğla", "Muş", "Nevşehir", "Niğde", "Ordu",
    "Osmaniye", "Rize", "Sakarya", "Samsun", "Siirt", "Sinop", "Sivas", "Şanlıurfa", "Şırnak", "Tekirdağ",
    "Tokat", "Trabzon", "Tunceli", "Uşak", "Van", "Yalova", "Yozgat", "Zonguldak"
]



def temizle_sehir(sehir):
    if pd.isna(sehir):
        return None
    if isinstance(sehir, str):
        sehir = sehir.lower()
        sehir = sehir.strip()

        for il in sehirler:
            if il.lower() in sehir:
                return il
    return None

In [59]:
train["Dogum Yeri Temizlenmiş"] = train["Dogum Yeri"].apply(temizle_sehir)
test["Dogum Yeri Temizlenmiş"] = test["Dogum Yeri"].apply(temizle_sehir)
train["Ikametgah Sehri Temizlenmiş"] = train["Ikametgah Sehri"].apply(temizle_sehir)
test["Ikametgah Sehri Temizlenmiş"] = test["Ikametgah Sehri"].apply(temizle_sehir)
train["Lise Sehir"]= train["Lise Sehir"].apply(temizle_sehir)
test["Lise Sehir"]= test["Lise Sehir"].apply(temizle_sehir)

In [60]:
train["Burs Aldigi Baska Kurum"].fillna("-", inplace=True)
test["Burs Aldigi Baska Kurum"].fillna("-", inplace=True)
train["Burs Aldigi Baska Kurum"] = train["Burs Aldigi Baska Kurum"].str.lower()
test["Burs Aldigi Baska Kurum"] = test["Burs Aldigi Baska Kurum"].str.lower()
train["burs_kategorisi"] = train["Burs Aldigi Baska Kurum"].apply(lambda x: 0 if x == "-" else (1 if "kyk" in x else 2))
test["burs_kategorisi"] = test["Burs Aldigi Baska Kurum"].apply(lambda x: 0 if x == "-" else (1 if "kyk" in x else 2))
train.drop(columns=['Burs Aldigi Baska Kurum'], inplace=True)
test.drop(columns=['Burs Aldigi Baska Kurum'], inplace=True)

In [61]:
top_categories = train["Bölüm"].value_counts().head(700).index

train["Bölüm"] = train["Bölüm"].apply(lambda x: x if x in top_categories else "Other")
test["Bölüm"] = test["Bölüm"].apply(lambda x: x if x in top_categories else "Other")

In [62]:
train['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'] = train['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'].fillna('-')
test['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'] = test['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'].fillna('-')
train['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'] = train['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'].apply(lambda x: 0 if x == '-' else 1)
test['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'] = test['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'].apply(lambda x: 0 if x == '-' else 1)

In [63]:
train['Burs Miktari'] = train['Baska Kurumdan Aldigi Burs Miktari'].apply(extract_numbers)
test['Burs Miktari'] = test['Baska Kurumdan Aldigi Burs Miktari'].apply(extract_numbers)

In [64]:
train['Degerlendirme Puani'] = train['Degerlendirme Puani'].fillna(0)
train['Dogum Yeri Temizlenmiş'] = train['Dogum Yeri Temizlenmiş'].fillna(0)
test['Dogum Yeri Temizlenmiş'] = test['Dogum Yeri Temizlenmiş'].fillna(0)
train['Ikametgah Sehri Temizlenmiş'] = train['Ikametgah Sehri Temizlenmiş'].fillna(0)
test['Ikametgah Sehri Temizlenmiş'] = test['Ikametgah Sehri Temizlenmiş'].fillna(0)

In [65]:
# def filling_scolar(data):
#     if 'Baska Bir Kurumdan Burs Aliyor mu?' in data.columns:
#         data['Baska Bir Kurumdan Burs Aliyor mu?'] = data['Baska Bir Kurumdan Burs Aliyor mu?'].apply(clean_text)
#         scolar_mapping = {
#             'evet': 1,
#             'hayır': 0,
#             'none': 0
#         }
#         data['Baska Bir Kurumdan Burs Aliyor mu?'] = data['Baska Bir Kurumdan Burs Aliyor mu?'].map(scolar_mapping)
#         missing_mask = data['Baska Bir Kurumdan Burs Aliyor mu?'].isna()
#         data.loc[missing_mask & data['Baska Kurumdan Aldigi Burs Miktari'].notna(), 'Baska Bir Kurumdan Burs Aliyor mu?'] = 1
#         mode_value = data['Baska Bir Kurumdan Burs Aliyor mu?'].mode()[0]
#         data['Baska Bir Kurumdan Burs Aliyor mu?'].fillna(mode_value, inplace=True)

#     if 'Baska Kurumdan Aldigi Burs Miktari' in data.columns:
#         data['Baska Kurumdan Aldigi Burs Miktari'] = data['Baska Kurumdan Aldigi Burs Miktari'].apply(extract_numbers)
#         data['Baska Kurumdan Aldigi Burs Miktari'] = data['Baska Kurumdan Aldigi Burs Miktari'].map(classify_amount)

#     return data

In [66]:
#MAPPINGS

cinsiyet_mapping = {'Erkek': 1, 'ERKEK': 1, 'Kadın': 2, 'Belirtmek istemiyorum': 0}
uni_mapping = {'Devlet': 1, 'DEVLET': 1, 'Özel': 0, 'ÖZEL': 0}
burs_mapping = {'evet': 1, 'Evet': 1, 'EVET': 1,'Hayır': 0, 'hayır': 0}
uni_sinif_mapping = {'1': 1, '2': 2, '3': 3,'4': 4, '5': 5,'6':6,'Yüksek Lisans':7,'tez':7,'Mezun':0,"Hazırlık":0,"hazırlık":0,"0":0}
uni_not_mapping = {'ORTALAMA BULUNMUYOR': 0, 'Not ortalaması yok': 0, 'Ortalama bulunmuyor': 0,'Hazırlığım': 0,
                   '0 - 1.79': 0,'1.00 - 2.50':1,'1.80 - 2.49':1,'2.50 ve altı':1,'2.00 - 2.50':1,"3.00-2.50":2,
                   "2.50 - 3.00":2,"2.50 - 2.99":2,'2.50 -3.00':2,'3.00 - 3.50':3,'3.50-3':3,'3.00 - 3.49':3,'3.00 - 4.00':3,'3.50 - 4.00':4,'4-3.5':4,'4.0-3.5':4}
mezun_mapping = {'Evet': 1, 'Hayır': 0}
lise_turu_mapping = {'İmam Hatip Lisesi': 0, 'Diğer': 0, 'Devlet': 0,'Düz lise': 0,
                   'Düz Lise': 0,'Meslek lisesi':1,'Meslek':1,'Meslek Lisesi':1,'Özel':1,"Özel Lisesi":1,
                   "Özel lisesi":1,"Özel Lise":1,'Anadolu Lisesi':2,'Anadolu lisesi':2,'Fen lisesi':3,'Fen Lisesi':3}

In [67]:
cinsiyet_mapping = {'Erkek': 1, 'ERKEK': 1, 'Kadın': 2, 'Belirtmek istemiyorum': 0}
uni_mapping = {'Devlet': 1, 'DEVLET': 1, 'Özel': 0, 'ÖZEL': 0}
burs_mapping = {'evet': 1, 'Evet': 1, 'EVET': 1,'Hayır': 0, 'hayır': 0}
uni_sinif_mapping = {'1': 1, '2': 2, '3': 3,'4': 4, '5': 5,'6':6,'Yüksek Lisans':7,'tez':7,'Mezun':0,"Hazırlık":0,"hazırlık":0,"0":0}
uni_not_mapping = {'ORTALAMA BULUNMUYOR': 0, 'Not ortalaması yok': 0, 'Ortalama bulunmuyor': 0,'Hazırlığım': 0,
                   '0 - 1.79': 0,'1.00 - 2.50':1,'1.80 - 2.49':1,'2.50 ve altı':1,'2.00 - 2.50':1,"3.00-2.50":2,
                   "2.50 - 3.00":2,"2.50 - 2.99":2,'2.50 -3.00':2,'3.00 - 3.50':3,'3.50-3':3,'3.00 - 3.49':3,'3.00 - 4.00':3,'3.50 - 4.00':4,'4-3.5':4,'4.0-3.5':4}
mezun_mapping = {'Evet': 1, 'Hayır': 0}
lise_turu_mapping = {'İmam Hatip Lisesi': 0, 'Diğer': 0, 'Devlet': 0,'Düz lise': 0,
                   'Düz Lise': 0,'Meslek lisesi':1,'Meslek':1,'Meslek Lisesi':1,'Özel':1,"Özel Lisesi":1,
                   "Özel lisesi":1,"Özel Lise":1,'Anadolu Lisesi':2,'Anadolu lisesi':2,'Fen lisesi':3,'Fen Lisesi':3}
lise_not_mapping = {
    '100 - 75': 5,
    '84 - 70': 4,
    '100 - 85': 5,
    '4.00 - 3.50': 5,
    '50 - 75': 3,
    '3.00 - 4.00': 4,
    '3.50 - 3.00': 4,
    '3.50 - 3': 4,
    '69 - 55': 3,
    '3.00 - 2.50': 3,
    '50 - 74': 3,
    '2.50 ve altı': 2,
    '54 - 45': 2,
    '25 - 50': 1,
    'Not ortalaması yok': 0,
    '44 - 0': 0,
    '0 - 25': 0,
    '25 - 49': 1,
    '0 - 24': 0
}
burslu_mapping = {'Evet': 1, 'Hayır': 0}
anne_egitim_mapping = {
    'İlkokul': 0,
    'İlkokul Mezunu': 0,
    'İLKOKUL MEZUNU': 0,
    'Eğitimi yok': 0,
    'EĞİTİM YOK': 0,
    'Eğitim Yok': 0,
    'Ortaokul': 1,
    'Ortaokul Mezunu': 1,
    'ORTAOKUL MEZUNU': 1,
    'Lise': 2,
    'LİSE': 2,
    'Lise Mezunu': 2,
    'Üniversite': 3,
    'ÜNİVERSİTE': 3,
    'Üniversite Mezunu': 3,
    'Yüksek Lisans': 4,
    'YÜKSEK LİSANS': 4,
    'Yüksek Lisans / Doktora': 4,
    'Yüksek Lisans / Doktara': 4,
    'Doktora': 5,
    'DOKTORA': 5
}
anne_calisma_mapping = {'Evet': 1, 'Hayır': 0,'Emekli':2}
anne_sector_mapping = {'0': 0, '-': 0,'Özel Sektör':1,'ÖZEL SEKTÖR':1,'Diğer':1,'DİĞER':1,'Kamu':2,'KAMU':2}
baba_egitim_mapping = {
    'İlkokul': 0,
    'İlkokul Mezunu': 0,
    'İLKOKUL MEZUNU': 0,
    'Eğitimi yok': 0,
    'EĞİTİM YOK': 0,
    'Eğitim Yok': 0,
    'Ortaokul': 1,
    'Ortaokul Mezunu': 1,
    'ORTAOKUL MEZUNU': 1,
    'Lise': 2,
    'LİSE': 2,
    'Lise Mezunu': 2,
    'Üniversite': 3,
    'ÜNİVERSİTE': 3,
    'Üniversite Mezunu': 3,
    'Yüksek Lisans': 4,
    'YÜKSEK LİSANS': 4,
    'Yüksek Lisans / Doktora': 4,
    'Yüksek Lisans / Doktara': 4,
    'Doktora': 5,
    'DOKTORA': 5,
    '0':0
}
baba_calisma_mapping = {'Evet': 1, 'Hayır': 0,'Emekli':2}
baba_sector_mapping = {'0': 0, '-': 0,'Özel Sektör':1,'ÖZEL SEKTÖR':1,'Diğer':1,'DİĞER':1,'Kamu':2,'KAMU':2}
kardes_sayisi_mapping = {
    'Kardeş Sayısı 1 Ek Bilgi Aile Hk. Anne Vefat': 100,
    0.0: 0,
    1.0: 1,
    2.0: 2,
    3.0: 3,
    4.0: 4,
    5.0: 5,
    6.0: 6,
    7.0: 7,
    8.0: 8,
    9.0: 9,
    10.0: 10,
    11.0: 11,
    12.0: 12,
    13.0: 13,
    14.0: 14,
    18: 18,
    0: 0,
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5,
    6: 6,
    7: 7,
    8: 8,
    9: 9,
    10: 10
}
giris_kulub_mapping = {'Hayır': 0, 'Evet': 1}
spor_mapping = {'Hayır': 0, 'Evet': 1}
spor_rol_mapping = {'0': 0, '-': 0,'Diğer':0,'DİĞER':0,'Bireysel':1,'Takım Oyuncusu':1,'Lider/Kaptan':2,'KAPTAN / LİDER':2,'Kaptan':2}
stk_mapping = {'Hayır': 0, 'Evet': 1}
stk2_mapping = {'Hayır': 0, 'Evet': 1}
girisim_mapping = {'Hayır': 0, 'Evet': 1}
ing_mapping = {'Hayır': 0, 'Evet': 1}
ing_seviye_mapping = {'0': 0, 'Başlangıç': 1,'Orta':2,'İleri':3}


In [68]:
def apply_mappings(train, test):
    train['Universite Kacinci Sinif'] = train['Universite Kacinci Sinif'].map(uni_sinif_mapping)
    test['Universite Kacinci Sinif'] = test['Universite Kacinci Sinif'].map(uni_sinif_mapping)

    train['Burs Aliyor mu?'] = train['Burs Aliyor mu?'].map(burs_mapping)
    test['Burs Aliyor mu?'] = test['Burs Aliyor mu?'].map(burs_mapping)

    train['Cinsiyet'] = train['Cinsiyet'].map(cinsiyet_mapping)
    test['Cinsiyet'] = test['Cinsiyet'].map(cinsiyet_mapping)

    train['Universite Turu'] = train['Universite Turu'].map(uni_mapping)
    test['Universite Turu'] = test['Universite Turu'].map(uni_mapping)

    train['Universite Not Ortalamasi'] = train['Universite Not Ortalamasi'].map(uni_not_mapping)
    test['Universite Not Ortalamasi'] = test['Universite Not Ortalamasi'].map(uni_not_mapping)

    train['Daha Once Baska Bir Universiteden Mezun Olmus'] = train['Daha Once Baska Bir Universiteden Mezun Olmus'].map(mezun_mapping)
    test['Daha Once Baska Bir Universiteden Mezun Olmus'] = test['Daha Once Baska Bir Universiteden Mezun Olmus'].map(mezun_mapping)

    train['Lise Turu'] = train['Lise Turu'].map(lise_turu_mapping)
    test['Lise Turu'] = test['Lise Turu'].map(lise_turu_mapping)

    train['Lise Mezuniyet Notu'] = train['Lise Mezuniyet Notu'].map(lise_not_mapping)
    test['Lise Mezuniyet Notu'] = test['Lise Mezuniyet Notu'].map(lise_not_mapping)

    train['Baska Bir Kurumdan Burs Aliyor mu?'] = train['Baska Bir Kurumdan Burs Aliyor mu?'].map(burslu_mapping)
    test['Baska Bir Kurumdan Burs Aliyor mu?'] = test['Baska Bir Kurumdan Burs Aliyor mu?'].map(burslu_mapping)

    train["Anne Egitim Durumu"] = train["Anne Egitim Durumu"].map(anne_egitim_mapping)
    test["Anne Egitim Durumu"] = test["Anne Egitim Durumu"].map(anne_egitim_mapping)

    train['Anne Calisma Durumu'] = train['Anne Calisma Durumu'].map(anne_calisma_mapping)
    test['Anne Calisma Durumu'] = test['Anne Calisma Durumu'].map(anne_calisma_mapping)

    train['Anne Sektor'] = train['Anne Sektor'].map(anne_sector_mapping)
    test['Anne Sektor'] = test['Anne Sektor'].map(anne_sector_mapping)

    train["Baba Egitim Durumu"] = train["Baba Egitim Durumu"].map(baba_egitim_mapping)
    test["Baba Egitim Durumu"] = test["Baba Egitim Durumu"].map(baba_egitim_mapping)

    train['Baba Calisma Durumu'] = train['Baba Calisma Durumu'].map(baba_calisma_mapping)
    test['Baba Calisma Durumu'] = test['Baba Calisma Durumu'].map(baba_calisma_mapping)

    train['Baba Sektor'] = train['Baba Sektor'].map(baba_sector_mapping)
    test['Baba Sektor'] = test['Baba Sektor'].map(baba_sector_mapping)

    train['Kardes Sayisi'] = train['Kardes Sayisi'].map(kardes_sayisi_mapping)
    test['Kardes Sayisi'] = test['Kardes Sayisi'].map(kardes_sayisi_mapping)

    train['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'] = train['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'].map(giris_kulub_mapping)
    test['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'] = test['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'].map(giris_kulub_mapping)

    train['Profesyonel Bir Spor Daliyla Mesgul musunuz?'] = train['Profesyonel Bir Spor Daliyla Mesgul musunuz?'].map(spor_mapping)
    test['Profesyonel Bir Spor Daliyla Mesgul musunuz?'] = test['Profesyonel Bir Spor Daliyla Mesgul musunuz?'].map(spor_mapping)

    train['Spor Dalindaki Rolunuz Nedir?'] = train['Spor Dalindaki Rolunuz Nedir?'].map(spor_rol_mapping)
    test['Spor Dalindaki Rolunuz Nedir?'] = test['Spor Dalindaki Rolunuz Nedir?'].map(spor_rol_mapping)

    train['Aktif olarak bir STK üyesi misiniz?'] = train['Aktif olarak bir STK üyesi misiniz?'].map(stk_mapping)
    test['Aktif olarak bir STK üyesi misiniz?'] = test['Aktif olarak bir STK üyesi misiniz?'].map(stk_mapping)

    train['Stk Projesine Katildiniz Mi?'] = train['Stk Projesine Katildiniz Mi?'].map(stk2_mapping)
    test['Stk Projesine Katildiniz Mi?'] = test['Stk Projesine Katildiniz Mi?'].map(stk2_mapping)

    train['Girisimcilikle Ilgili Deneyiminiz Var Mi?'] = train['Girisimcilikle Ilgili Deneyiminiz Var Mi?'].map(girisim_mapping)
    test['Girisimcilikle Ilgili Deneyiminiz Var Mi?'] = test['Girisimcilikle Ilgili Deneyiminiz Var Mi?'].map(girisim_mapping)

    train['Ingilizce Biliyor musunuz?'] = train['Ingilizce Biliyor musunuz?'].map(ing_mapping)
    test['Ingilizce Biliyor musunuz?'] = test['Ingilizce Biliyor musunuz?'].map(ing_mapping)

    train['Ingilizce Seviyeniz?'] = train['Ingilizce Seviyeniz?'].map(ing_seviye_mapping)
    test['Ingilizce Seviyeniz?'] = test['Ingilizce Seviyeniz?'].map(ing_seviye_mapping)

    columns_to_drop = ['Lise Adi', 'Lise Adi Diger', 'Dogum Tarihi', 'Lise Bolumu', 'Lise Bolum Diger', 'Baska Kurumdan Aldigi Burs Miktari', 'Dogum Yeri', 'Ikametgah Sehri']
    train.drop(columns=columns_to_drop, inplace=True, errors='ignore')
    test.drop(columns=columns_to_drop, inplace=True, errors='ignore')

    le = LabelEncoder()
    label_encoders = {}

    for column in ['Dogum Yeri Temizlenmiş', 'Ikametgah Sehri Temizlenmiş', 'Lise Sehir', 'Cinsiyet', 'Universite Adi', 'Universite Turu', 'Uye Oldugunuz Kulubun Ismi', "Hangi STK'nin Uyesisiniz?", 'Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite', 'Bölüm']:
        train[column] = train[column].astype(str)
        test[column] = test[column].astype(str)

        le = LabelEncoder()
        le.fit(train[column])
        label_encoders[column] = le

        # Transform train and test, using .transform() for test and handling unseen labels
        train[column] = le.transform(train[column])
        test[column] = test[column].apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)

    scaler = StandardScaler()
    columns_to_scale = ['Dogum Yeri Temizlenmiş', 'Ikametgah Sehri Temizlenmiş', 'Lise Sehir', 'Cinsiyet', 'Universite Adi', 'Universite Turu', 'Uye Oldugunuz Kulubun Ismi', "Hangi STK'nin Uyesisiniz?", 'Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite', 'Bölüm']
    train[columns_to_scale] = scaler.fit_transform(train[columns_to_scale])
    test[columns_to_scale] = scaler.transform(test[columns_to_scale])

    train.fillna(0, inplace=True)
    test.fillna(0, inplace=True)

    return train, test

In [71]:
train, test = apply_mappings(train, test)
train.head()

,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,Bölüm,Universite Kacinci Sinif,Universite Not Ortalamasi,Daha Once Baska Bir Universiteden Mezun Olmus,Lise Sehir,Lise Turu,Lise Mezuniyet Notu,Baska Bir Kurumdan Burs Aliyor mu?,Anne Egitim Durumu,Anne Calisma Durumu,Anne Sektor,Baba Egitim Durumu,Baba Calisma Durumu,Baba Sektor,Kardes Sayisi,Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?,Uye Oldugunuz Kulubun Ismi,Profesyonel Bir Spor Daliyla Mesgul musunuz?,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id,Dogum Yili,Basvuru Yasi,Dogum Yeri Temizlenmiş,Ikametgah Sehri Temizlenmiş,burs_kategorisi,Burs Miktari
0,2014,52.0,-0.975106,1.200203,-2.119428,100.0,1,-1.615500,3.0,3.0,0.0,-1.549798,2.0,0.0,0.0,3.0,0.0,0.0,3.0,2.0,0.0,0.0,1.0,-3.135698,1.0,0.0,0.0,0.433309,1.0,0.0,0,0.0,0.0,0.062255,0,1994.0,20.0,-1.124230,-1.159097,0,0
1,2014,30.0,-0.975106,1.200203,-2.119428,100.0,0,-1.202899,3.0,2.0,0.0,-1.549798,2.0,4.0,0.0,3.0,1.0,2.0,4.0,2.0,0.0,0.0,0.0,0.421774,0.0,0.0,0.0,0.433309,1.0,1.0,1,0.0,0.0,0.062255,1,1993.0,21.0,-1.329632,1.140870,0,0
2,2014,18.0,-0.975106,0.848958,-2.119428,100.0,0,-0.007965,1.0,0.0,0.0,-1.549798,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,-0.513054,0.0,0.0,0.0,0.433309,0.0,0.0,0,0.0,0.0,0.062255,2,1986.0,28.0,0.792853,1.140870,0,0
3,2014,40.0,-0.975106,1.416354,-2.119428,100.0,1,-1.604783,3.0,2.0,0.0,-0.893340,3.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,1.0,-2.061019,0.0,0.0,0.0,0.433309,1.0,0.0,0,0.0,0.0,0.062255,3,1991.0,23.0,-0.542259,1.140870,0,0
4,2014,24.0,-0.975106,0.740882,-2.119428,100.0,1,-0.007965,0.0,2.0,0.0,-1.549798,1.0,4.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.421774,0.0,0.0,0.0,0.433309,0.0,0.0,0,0.0,0.0,0.062255,4,1992.0,22.0,-1.124230,-1.159097,0,0


Datasets are ready for modelling

In [70]:
train.to_csv("train_processed.csv", index=False)
test.to_csv("test_processed.csv", index=False)